# TensorFlow and TensorBoard with Regularization



## Purpose

The purpose of this lab is threefold.  

1.   to review using `TensorFlow` and `TensorBoard` for modeling and evaluation with neural networks
2.   to review using data science pipelines and cross-validation with neural networks
3.   to review using `TensorFlow` for neural network regularization

We'll be continuting our investigation of the canonical [Titanic Data Set](https://www.kaggle.com/competitions/titanic/overview) that we began [previously](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

## The Titanic

### The Titanic and it's data



RMS Titanic was a British passenger liner built by Harland and Wolf and operated by the White Star Line. It sank in the North Atlantic Ocean in the early morning hours of 15 April 1912, after striking an iceberg during her maiden voyage from Southampton, England to New York City, USA.

Of the estimated 2,224 passengers and crew aboard, more than 1,500 died, making the sinking one of modern history's deadliest peacetime commercial marine disasters. 

Though there were about 2,224 passengers and crew members, we are given data of about 1,300 passengers. Out of these 1,300 passengers details, about 900 data is used for training purpose and remaining 400 is used for test purpose. The test data has had the survived column removed and we'll use neural networks to predict whether the passengers in the test data survived or not. Both training and test data are not perfectly clean as we'll see.

Below is a picture of the Titanic Museum in Belfast, Northern Ireland.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://upload.wikimedia.org/wikipedia/commons/c/c0/Titanic_Belfast_HDR.jpg", width=400, height=400)

### Data Dictionary

*   *Survival* : 0 = No, 1 = Yes
*   *Pclass* : A proxy for socio-economic status (SES)
  *   1st = Upper
  *   2nd = Middle
  *   3rd = Lower
*   *sibsp* : The number of siblings / spouses aboard the Titanic
  *   Sibling = brother, sister, stepbrother, stepsister
  *   Spouse = husband, wife (mistresses and fiancés were ignored)
*   *parch* : The # of parents / children aboard the Titanic
  *   Parent = mother, father
  *   Child = daughter, son, stepdaughter, stepson
  *   Some children travelled only with a nanny, therefore *parch*=0 for them.
*   *Ticket* : Ticket number
*   *Fare* : Passenger fare (British pounds)
*   *Cabin* : Cabin number embarked
*   *Embarked* : Port of Embarkation
  *   C = Cherbourg (now Cherbourg-en-Cotentin), France
  *   Q = Queenstown (now Cobh), Ireland
  *   S = Southampton, England
*   *Name*, *Sex*, *Age* (years) are all self-explanatory

## Libraries and the Data



### Importing libraries

In [11]:
# Load the germane libraries

import pandas as pd
import numpy as np
import seaborn as sns 
from pandas._libs.tslibs import timestamps
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import keras 
from keras import models
from sklearn.impute import SimpleImputer
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.losses import binary_crossentropy
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier

# Load the TensorBoard notebook extension and related libraries
%load_ext tensorboard
import datetime

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Loading the data

In [12]:
# Load the data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# We need to do this for when we mamke our predictions from the test data at the end
ids = test[['PassengerId']]

## EDA and Preprocessing

### Exploratory Data Analysis

You have already performed EDA on this data set. Look back on what you did before or see [here](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

Of course, feel free to re-run what you have done before or try out some other EDA as you find useful.

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
print("Train rows", len(train))
print("Test rows", len(test))
print("Train columns", len(train.columns))
print("Test columns", len(test.columns))

Train rows 891
Test rows 418
Train columns 12
Test columns 11


### Preprocessing

Let's do the same prepricessing as before.

In [13]:
# Performing preprocessing on the train and test data will be more effecient if we combine the two date sets.
combined = pd.concat([train, test], axis=0, sort=False)

#Age column
combined['Age'].fillna(combined['Age'].median(),inplace=True) # Age

# Embarked column
combined['Embarked'].fillna(combined['Embarked'].value_counts().index[0], inplace=True) # Embarked
combined['Fare'].fillna(combined['Fare'].median(),inplace=True)

# Class column
d = {1:'1st',2:'2nd',3:'3rd'} #Pclass
combined['Pclass'] = combined['Pclass'].map(d) #Pclass

# Making Age into adult (1) and child (0)
combined['Child'] = combined['Age'].apply(lambda age: 1 if age>=18 else 0) 

# Break up the string that has the title and names
combined['Title'] = combined['Name'].str.split('.').str.get(0)  # output : 'Futrelle, Mrs'
combined['Title'] = combined['Title'].str.split(',').str.get(1) # output : 'Mrs '
combined['Title'] = combined['Title'].str.strip()               # output : 'Mrs'
combined.groupby('Title').count()

# Replace the French titles with Enlgish
french_titles = ['Don', 'Dona', 'Mme', 'Ms', 'Mra','Mlle']
english_titles = ['Mr', 'Mrs','Mrs','Mrs','Mrs','Miss']
for i in range(len(french_titles)):
    for j in range(len(english_titles)):
        if i == j:
            combined['Title'] = combined['Title'].str.replace(french_titles[i],english_titles[j])

# Seperate the titles into "major" and "others", the latter would be, e.g., Reverend
major_titles = ['Mr','Mrs','Miss','Master']
combined['Title'] = combined['Title'].apply(lambda title: title if title in major_titles else 'Others')

#Dropping the Irrelevant Columns
combined.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

# Getting Dummy Variables and Dropping the Original Categorical Variables
categorical_vars = combined[['Pclass','Sex','Embarked','Title','Child']] # Get Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
combined = combined.drop(['Pclass','Sex','Embarked','Title','Child'],axis=1)
combined = pd.concat([combined, dummies],axis=1)

# Separating the data back into train and test sets
test = combined[combined['Survived'].isnull()].drop(['Survived'],axis=1)
train = combined[combined['Survived'].notnull()]

# Training
X_train = train.drop(['Survived'],axis=1)
y_train = train['Survived']

# Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
test = sc.fit_transform(test)

## Neural Network Model

### Building the model

#### Define the model as a pipeline

Let's use the data science pipeline for our neural network model.

As you are now using regularization to guard against high variance, i.e. overfitting the data, in the definition of the model below include *dropout* and/or *l2* regularization. Also, feel free to experiment with different activation functions.

In [15]:
# It will help to define our model in terms of a pipeline
def build_classifier(optimizer):
# insert Sequential and layers here
    classifier = Sequential()
    classifier.add(Dense(50,input_shape=(X_train.shape[1],),activation='sigmoid'))
    classifier.add(Dense(20,activation='relu'))
    classifier.add(Dense(1,activation='sigmoid'))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

#### Use grid search to find help you tune the parameters

You can play with optimizers, epochs, and batch sizes. The ones that we're suggesting are not necessarily the best.

In [16]:
# Grid Search
classifier = KerasClassifier(build_fn = build_classifier)
param_grid = dict(optimizer = ['Adam'],
                  epochs=[10, 20, 50],
                  batch_size=[16, 25, 32])
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

Epoch 1/10
45/45 [==============================] - 0s 4ms/step - loss: 0.6422 - accuracy: 0.6166
Epoch 2/10
45/45 [==============================] - 0s 3ms/step - loss: 0.5575 - accuracy: 0.7767
Epoch 3/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4996 - accuracy: 0.8034
Epoch 4/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4656 - accuracy: 0.7921
Epoch 5/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4487 - accuracy: 0.8076
Epoch 6/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4399 - accuracy: 0.8076
Epoch 7/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4319 - accuracy: 0.8174
Epoch 8/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4249 - accuracy: 0.8216
Epoch 9/10
45/45 [==============================] - 0s 4ms/step - loss: 0.4241 - accuracy: 0.8230
Epoch 10/10
45/45 [==============================] - 0s 4ms/step - loss: 0.4196 - accuracy: 0.8301
Instructions for up

45/45 [==============================] - 0s 4ms/step - loss: 0.4330 - accuracy: 0.8163
Epoch 8/20
45/45 [==============================] - 0s 3ms/step - loss: 0.4275 - accuracy: 0.8219
Epoch 9/20
45/45 [==============================] - 0s 4ms/step - loss: 0.4229 - accuracy: 0.8233
Epoch 10/20
45/45 [==============================] - 0s 3ms/step - loss: 0.4195 - accuracy: 0.8261
Epoch 11/20
45/45 [==============================] - 0s 3ms/step - loss: 0.4172 - accuracy: 0.8289
Epoch 12/20
45/45 [==============================] - 0s 4ms/step - loss: 0.4152 - accuracy: 0.8275
Epoch 13/20
45/45 [==============================] - 0s 3ms/step - loss: 0.4139 - accuracy: 0.8219
Epoch 14/20
45/45 [==============================] - 0s 3ms/step - loss: 0.4125 - accuracy: 0.8303
Epoch 15/20
45/45 [==============================] - 0s 4ms/step - loss: 0.4084 - accuracy: 0.8345
Epoch 16/20
45/45 [==============================] - 0s 3ms/step - loss: 0.4091 - accuracy: 0.8275
Epoch 17/20
45/45 [=====

45/45 [==============================] - 0s 2ms/step - loss: 0.4194 - accuracy: 0.8230
Epoch 10/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4145 - accuracy: 0.8287
Epoch 11/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4127 - accuracy: 0.8301
Epoch 12/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4140 - accuracy: 0.8230
Epoch 13/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4092 - accuracy: 0.8272
Epoch 14/50
45/45 [==============================] - 0s 4ms/step - loss: 0.4073 - accuracy: 0.8343
Epoch 15/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4048 - accuracy: 0.8258
Epoch 16/50
45/45 [==============================] - 0s 3ms/step - loss: 0.4069 - accuracy: 0.8301
Epoch 17/50
45/45 [==============================] - 0s 2ms/step - loss: 0.4070 - accuracy: 0.8287
Epoch 18/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3970 - accuracy: 0.8301
Epoch 19/50
45/45 [===

45/45 [==============================] - 0s 3ms/step - loss: 0.3964 - accuracy: 0.8373
Epoch 42/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3985 - accuracy: 0.8317
Epoch 43/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3954 - accuracy: 0.8331
Epoch 44/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3967 - accuracy: 0.8401
Epoch 45/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3924 - accuracy: 0.8429
Epoch 46/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3891 - accuracy: 0.8457
Epoch 47/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3915 - accuracy: 0.8359
Epoch 48/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3905 - accuracy: 0.8443
Epoch 49/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3889 - accuracy: 0.8415
Epoch 50/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3870 - accuracy: 0.8373
Epoch 1/50
45/45 [====

45/45 [==============================] - 0s 3ms/step - loss: 0.3921 - accuracy: 0.8373
Epoch 24/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3950 - accuracy: 0.8387
Epoch 25/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3915 - accuracy: 0.8387
Epoch 26/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3935 - accuracy: 0.8373
Epoch 27/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3912 - accuracy: 0.8513
Epoch 28/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3905 - accuracy: 0.8415
Epoch 29/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3915 - accuracy: 0.8471
Epoch 30/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3881 - accuracy: 0.8401
Epoch 31/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3872 - accuracy: 0.8485
Epoch 32/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3873 - accuracy: 0.8401
Epoch 33/50
45/45 [===

...

23/23 [==============================] - 0s 2ms/step - loss: 0.4420 - accuracy: 0.8107
Epoch 8/20
23/23 [==============================] - 0s 4ms/step - loss: 0.4404 - accuracy: 0.8135
Epoch 9/20
23/23 [==============================] - 0s 2ms/step - loss: 0.4322 - accuracy: 0.8177
Epoch 10/20
23/23 [==============================] - ETA: 0s - loss: 0.4270 - accuracy: 0.82 - 0s 3ms/step - loss: 0.4283 - accuracy: 0.8177
Epoch 11/20
23/23 [==============================] - 0s 2ms/step - loss: 0.4265 - accuracy: 0.8261
Epoch 12/20
23/23 [==============================] - 0s 2ms/step - loss: 0.4231 - accuracy: 0.8205
Epoch 13/20
23/23 [==============================] - 0s 3ms/step - loss: 0.4208 - accuracy: 0.8261
Epoch 14/20
23/23 [==============================] - 0s 3ms/step - loss: 0.4196 - accuracy: 0.8275
Epoch 15/20
23/23 [==============================] - 0s 2ms/step - loss: 0.4166 - accuracy: 0.8261
Epoch 16/20
23/23 [==============================] - 0s 2ms/step - loss: 0.4160 -

23/23 [==============================] - 0s 1ms/step - loss: 0.4016 - accuracy: 0.8287
Epoch 30/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4025 - accuracy: 0.8301
Epoch 31/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4016 - accuracy: 0.8371
Epoch 32/50
23/23 [==============================] - 0s 2ms/step - loss: 0.3980 - accuracy: 0.8315
Epoch 33/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4012 - accuracy: 0.8371
Epoch 34/50
23/23 [==============================] - 0s 2ms/step - loss: 0.3978 - accuracy: 0.8371
Epoch 35/50
23/23 [==============================] - 0s 2ms/step - loss: 0.3985 - accuracy: 0.8427
Epoch 36/50
23/23 [==============================] - 0s 2ms/step - loss: 0.3968 - accuracy: 0.8329
Epoch 37/50
23/23 [==============================] - 0s 2ms/step - loss: 0.3968 - accuracy: 0.8385
Epoch 38/50
23/23 [==============================] - 0s 2ms/step - loss: 0.3927 - accuracy: 0.8469
Epoch 39/50
23/23 [===

23/23 [==============================] - 0s 2ms/step - loss: 0.4302 - accuracy: 0.8163
Epoch 12/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4285 - accuracy: 0.8205
Epoch 13/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4269 - accuracy: 0.8261
Epoch 14/50
23/23 [==============================] - 0s 3ms/step - loss: 0.4237 - accuracy: 0.8247
Epoch 15/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4227 - accuracy: 0.8247
Epoch 16/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4228 - accuracy: 0.8205
Epoch 17/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4269 - accuracy: 0.8275
Epoch 18/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4257 - accuracy: 0.8177
Epoch 19/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4186 - accuracy: 0.8303
Epoch 20/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4195 - accuracy: 0.8317
Epoch 21/50
23/23 [===

Epoch 44/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3905 - accuracy: 0.8401
Epoch 45/50
23/23 [==============================] - 0s 2ms/step - loss: 0.3886 - accuracy: 0.8499
Epoch 46/50
23/23 [==============================] - 0s 2ms/step - loss: 0.3911 - accuracy: 0.8415
Epoch 47/50
23/23 [==============================] - 0s 3ms/step - loss: 0.3883 - accuracy: 0.8471
Epoch 48/50
23/23 [==============================] - 0s 3ms/step - loss: 0.3862 - accuracy: 0.8457
Epoch 49/50
23/23 [==============================] - 0s 3ms/step - loss: 0.3869 - accuracy: 0.8443
Epoch 50/50
23/23 [==============================] - 0s 3ms/step - loss: 0.3856 - accuracy: 0.8527
Epoch 1/50
23/23 [==============================] - 0s 4ms/step - loss: 0.7425 - accuracy: 0.4951
Epoch 2/50
23/23 [==============================] - 0s 3ms/step - loss: 0.6160 - accuracy: 0.6732
Epoch 3/50
23/23 [==============================] - 0s 3ms/step - loss: 0.5797 - accuracy: 0.7532
Epoch 4/50
23

56/56 [==============================] - 0s 3ms/step - loss: 0.3994 - accuracy: 0.8395
Epoch 27/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4002 - accuracy: 0.8384
Epoch 28/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3991 - accuracy: 0.8339
Epoch 29/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3965 - accuracy: 0.8440: 0s - loss: 0.4146 - accuracy: 0.
Epoch 30/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3964 - accuracy: 0.8316
Epoch 31/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3990 - accuracy: 0.8350
Epoch 32/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3955 - accuracy: 0.8350
Epoch 33/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3946 - accuracy: 0.8395
Epoch 34/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3938 - accuracy: 0.8395
Epoch 35/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3945 - accur

#### `TensorBoard`

`TensorBoard` is `TensorFlow`'s visualization toolkit. It is a dashboard that provides visualization and tooling that is needed for machine learning experimentation. The code immediately below will allow us to use TensorBoard.

N.B. When we loaded the libraries, we loaded the TensorBoard notebook extension. (It is the last line of code in the first code chunk.)

In [18]:
# Clear out any prior log data.
!rm -rf logs
# Be careful not to run this command if already have trained your model and you want to use TensorBoard.

# Sets up a timestamped log directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(log_dir)


# The callback function, which will be called in the fit()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fitting the optimal model and evaluating with `TensorBoaard`

Define the early stopping callback. Use your best values from grid serarch with `KerasClassifer` and finally fit the model.

In [19]:
# Define the EarlyStopping object
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-8,
                           verbose=1, patience=5,
                           mode='min')

# Using KerasClassifier
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer=best_parameters['optimizer'],
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

# Fit the model with the tensorboard_callback
classifier.fit(X_train,
               y_train,
               verbose=1,
               callbacks=[early_stop, tensorboard_callback])


# Warning: If verbose = 0 (silent) or 2 (one line per epoch), then on TensorBoard's Graphs tab there will be an error.
# The other tabs in TensorBoard will still be function, but if you want the graphs then verbose needs to be 1 (progress bar).

Epoch 1/50
 1/56 [..............................] - ETA: 0s - loss: 0.7511 - accuracy: 0.3750WARNING:tensorflow:From C:\Users\nisalas\AppData\Local\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
56/56 [==============================] - 0s 7ms/step - loss: 0.5953 - accuracy: 0.6801
Epoch 2/50
56/56 [==============================] - 0s 3ms/step - loss: 0.5175 - accuracy: 0.7868
Epoch 3/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4695 - accuracy: 0.8002
Epoch 4/50
56/56 [==============================] - 0s 2ms/step - loss: 0.4547 - accuracy: 0.8058
Epoch 5/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4420 - accuracy: 0.8103
Epoch 6/50
56/56 [==============================] - 0s 2ms/step - loss: 0.4384 - accuracy: 0.8126
Epoch 7/50
56/56 

56/56 [==============================] - 0s 3ms/step - loss: 0.4054 - accuracy: 0.8238
Epoch 26/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4057 - accuracy: 0.8328
Epoch 27/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4045 - accuracy: 0.8316
Epoch 28/50
56/56 [==============================] - 0s 4ms/step - loss: 0.4019 - accuracy: 0.8361
Epoch 29/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4004 - accuracy: 0.8373
Epoch 30/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4007 - accuracy: 0.8316
Epoch 31/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3994 - accuracy: 0.8361
Epoch 32/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3999 - accuracy: 0.8316
Epoch 33/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3989 - accuracy: 0.8384
Epoch 34/50
56/56 [==============================] - 0s 2ms/step - loss: 0.3967 - accuracy: 0.8384
Epoch 35/50
56/56 [===

In [20]:
# Call TensorBoard within SaturnCloud [Comment this out if you are not in SaturnCloud]
import os
print(f"https://{os.getenv('SATURN_JUPYTER_BASE_DOMAIN')}/proxy/8000/")
%tensorboard --logdir logs/fit --port 8000 --bind_all
# This will generate a hyperlink. Click on that to open TensorBoard!
# (You'll see a 404 error below the link, just ignore that.)

# Call TensorBoard [Not in SaturnCloud]
# Uncomment the next time if you are not in SC
%tensorboard --logdir logs/fit

https://None/proxy/8000/


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 32812.

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 40364.

#### Results and Predictions

Calculate the predictions, save them as a csv, and print them.

In [ ]:
# Your code here (use more cells if you need to)


Continue to tweak your model until you are happy with the results based on model evaluation.

## Conclusion

Now that you have the `TensorBoard` to help you look at your model, you can better understand how to tweak your model.

How do your predictions compare to what you did last time?

Remember that your "fancier" model may be less accurate... but that is okay if that is the case since we're trying to guard against variance with regularization techniques.